In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

In [2]:
code = '035420'

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36 Edg/101.0.1210.32'}

# 마지막 페이지 체크
last_page = 1
while True:
    try:
        url = f'https://finance.naver.com/item/news_news.naver?code={code}&page={last_page}&sm=title_entity_id.basic&clusterId='
        res = requests.get(url, headers = headers)
        bs = BeautifulSoup(res.text, 'html.parser')
        
        last_page_src = bs.find_all('td',{'class':'pgRR'})[0].find('a')['href']
        last_page = re.search('page=[0-9]+',last_page_src).group().replace('page=','')
        last_page = int(last_page)
    except:
        break
print(f'마지막 페이지:{last_page}')

date_list = []
title_list = []
info_list = []
url_list = []

sub_date_list = []
sub_title_list = []
sub_info_list = []
sub_url_list = []

for page in range(1,last_page+1):
    url = f'https://finance.naver.com/item/news_news.naver?code={code}&page={page}&sm=title_entity_id.basic&clusterId='
    res = requests.get(url, headers = headers)
    bs = BeautifulSoup(res.text, 'html.parser')
    
    # 뉴스 전체 크롤링
    art_cnt = len(bs.find_all('td',{'class':'title'}))
    for i in range(art_cnt):
        date_list.append(bs.find_all('td',{'class':'date'})[i].text.split()[0].replace('.','-'))
        title_list.append(bs.find_all('td',{'class':'title'})[i].text.replace('\n',''))
        info_list.append(bs.find_all('td',{'class':'info'})[i].text)
        url_list.append('https://finance.naver.com'+bs.find_all('td',{'class':'title'})[i].find('a')['href'])
    
    # 관련 뉴스만 크롤링
    chk = 1
    while True:
        try:
            chk_len = len(bs.find_all('table',{'class':'type5'})[chk].find_all('td',{'class':'title'}))
            for i in range(chk_len):
                sub_date_list.append(bs.find_all('table',{'class':'type5'})[chk].find_all('td',{'class':'date'})[i].text.split()[0].replace('.','-'))
                sub_title_list.append(bs.find_all('table',{'class':'type5'})[chk].find_all('td',{'class':'title'})[i].text.replace('\n',''))
                sub_info_list.append(bs.find_all('table',{'class':'type5'})[chk].find_all('td',{'class':'info'})[i].text)
                sub_url_list.append('https://finance.naver.com'+bs.find_all('table',{'class':'type5'})[chk].find_all('td',{'class':'title'})[i].find('a')['href'])
            chk += 1
        except:
            break   
  
    if page % 10 == 0:
        print(f'현재 {page}페이지 크롤링 완료')

마지막 페이지:358
현재 10페이지 크롤링 완료
현재 20페이지 크롤링 완료
현재 30페이지 크롤링 완료
현재 40페이지 크롤링 완료
현재 50페이지 크롤링 완료
현재 60페이지 크롤링 완료
현재 70페이지 크롤링 완료
현재 80페이지 크롤링 완료
현재 90페이지 크롤링 완료
현재 100페이지 크롤링 완료
현재 110페이지 크롤링 완료
현재 120페이지 크롤링 완료
현재 130페이지 크롤링 완료
현재 140페이지 크롤링 완료
현재 150페이지 크롤링 완료
현재 160페이지 크롤링 완료
현재 170페이지 크롤링 완료
현재 180페이지 크롤링 완료
현재 190페이지 크롤링 완료
현재 200페이지 크롤링 완료
현재 210페이지 크롤링 완료
현재 220페이지 크롤링 완료
현재 230페이지 크롤링 완료
현재 240페이지 크롤링 완료
현재 250페이지 크롤링 완료
현재 260페이지 크롤링 완료
현재 270페이지 크롤링 완료
현재 280페이지 크롤링 완료
현재 290페이지 크롤링 완료
현재 300페이지 크롤링 완료
현재 310페이지 크롤링 완료
현재 320페이지 크롤링 완료
현재 330페이지 크롤링 완료
현재 340페이지 크롤링 완료
현재 350페이지 크롤링 완료


In [3]:
df = pd.DataFrame()
df['날짜'] = date_list
df['제목'] = title_list
df['정보제공'] = info_list
df['링크'] = url_list
df

,날짜,제목,정보제공,링크
0,2022-05-10,"네이버, AI 프로젝트 챌린지 참가자 모집한다",파이낸셜뉴스,https://finance.naver.com/item/news_read.naver...
1,2022-05-10,"네이버, AI 프로젝트 챌린지 열어…서비스 기획 분야도 신설",이데일리,https://finance.naver.com/item/news_read.naver...
2,2022-05-10,"레드아이스·부천시, 네이버웹툰과 총상금 1억 5000만 원 웹툰 공모전...",이데일리,https://finance.naver.com/item/news_read.naver...
3,2022-05-09,성장주 카카오·네이버의 추락…주가 반토막에 증권가 전망도 ‘흐림’,조선비즈,https://finance.naver.com/item/news_read.naver...
4,2022-05-09,"[단독] 네이버, 웹3 스타트업 등에 1000억 베팅",서울경제,https://finance.naver.com/item/news_read.naver...
...,...,...,...,...
5694,2021-05-10,"“연구진 100명 투입”…네이버, 서울대와 ‘초대규모’ AI 연구센터 ...",헤럴드경제,https://finance.naver.com/item/news_read.naver...
5695,2021-05-10,"네이버, 서울대와 손잡고 AI 연구센터 설립…연구진 100명 투입",이데일리,https://finance.naver.com/item/news_read.naver...
5696,2021-05-10,"네이버, 서울대와 수백억 규모 AI 연구센터 세운다",파이낸셜뉴스,https://finance.naver.com/item/news_read.naver...
5697,2021-05-10,네이버-서울대 '초대규모 AI 연구센터' 설립…연구진 100여명 투입,머니투데이,https://finance.naver.com/item/news_read.naver...


In [4]:
df2 = pd.DataFrame() # 관련기사
df2['날짜'] = sub_date_list
df2['제목'] = sub_title_list
df2['정보제공'] = sub_info_list
df2['링크'] = sub_url_list
df2

,날짜,제목,정보제공,링크
0,2022-05-10,"네이버, AI 프로젝트 챌린지 열어…서비스 기획 분야도 신설",이데일리,https://finance.naver.com/item/news_read.naver...
1,2022-05-06,'억대' 출연료 네이버가 만든 이 여성…목소리 컬투쇼에 나온다,서울경제,https://finance.naver.com/item/news_read.naver...
2,2022-05-06,"가상인간 ‘로지’ AI 목소리, 네이버가 만들었다",이데일리,https://finance.naver.com/item/news_read.naver...
3,2022-05-05,"어바웃펫, 네이버로부터 100억원 투자 유치…“오픈마켓 선점·IT 기...",아시아경제,https://finance.naver.com/item/news_read.naver...
4,2022-05-04,"주3일 출근? 원격근무?…네이버, 직원 자율성 강화 근무제 도입",이데일리,https://finance.naver.com/item/news_read.naver...
...,...,...,...,...
2114,2021-05-10,"네이버, 서울대와 손잡고 AI 연구센터 설립…연구진 100명 투입",이데일리,https://finance.naver.com/item/news_read.naver...
2115,2021-05-10,"네이버, 서울대와 수백억 규모 AI 연구센터 세운다",파이낸셜뉴스,https://finance.naver.com/item/news_read.naver...
2116,2021-05-10,"네이버, 1769억원 규모 자기주식 처분 결정",이데일리,https://finance.naver.com/item/news_read.naver...
2117,2021-05-10,네이버-서울대 '초대규모 AI 연구센터' 설립…연구진 100여명 투입,머니투데이,https://finance.naver.com/item/news_read.naver...


In [5]:
print(df.shape[0]-df2.shape[0]) # 페이지수 * 10

3580


In [6]:
df.to_csv("./src/네이버금융종목뉴스링크_네이버_전체.csv", index=False)
df2.to_csv("./src/네이버금융종목뉴스링크_네이버_관련뉴스.csv", index=False)